In [2]:
%load_ext tensorboard
%tensorboard --logdir logs

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_addons as tfa
from numpy.random import default_rng
rng = default_rng()


In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

AttributeError: module 'tensorflow' has no attribute 'Session'

In [39]:
SEED = 42
PROJECT_FOLDER = "../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
TRAIN_DATA_FOLDER = PROJECT_FOLDER + "train_images/"
TEST_DATA_FOLDER = PROJECT_FOLDER + "test_images/"

IMAGE_SIZE = (380, 380)
VAL_SPLIT = 0.1

In [40]:
train_ds = image_dataset_from_directory(
    directory=TRAIN_DATA_FOLDER,
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    seed=1337,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True,
    subset='training',
    validation_split=VAL_SPLIT
)

val_ds = image_dataset_from_directory(
    directory=TRAIN_DATA_FOLDER,
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    seed=1337,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True,
    subset='validation',
    validation_split=VAL_SPLIT
)

test_ds = image_dataset_from_directory(
    directory=TEST_DATA_FOLDER,
    labels=None,
    batch_size=1,
    seed=1337,
    image_size=IMAGE_SIZE,
    crop_to_aspect_ratio=True,
)

Found 2081 files belonging to 100 classes.
Using 1873 files for training.
Found 2081 files belonging to 100 classes.
Using 208 files for validation.
Found 1 files belonging to 1 classes.


In [41]:
n_classes = len(train_ds.class_names)
n_classes

100

In [42]:
def random_cutout(images_tensor, labels):
    ratios_x = rng.uniform(0.2, 0.6)*images_tensor.shape[1]
    ratios_y = rng.uniform(0.2, 0.6)*images_tensor.shape[2]

    xs_mask = tf.cast(ratios_x, tf.int32)
    xs_mask = xs_mask if xs_mask % 2 == 0 else xs_mask+1 # force number even

    ys_mask = tf.cast(ratios_y, tf.int32)
    ys_mask = ys_mask if ys_mask % 2 == 0 else ys_mask+1 # force number even

    return tfa.image.random_cutout(images_tensor, (xs_mask, ys_mask), constant_values = (255.0, 0.0, 0.0)), labels

train_ds = train_ds.map(random_cutout)
val_ds = val_ds.map(random_cutout)

# Models

In [43]:

def create_baseline_efficientnet_model(n_classes):
    efficientnet_layer = tf.keras.applications.EfficientNetB4(
            include_top=True,
            weights="imagenet")
    efficientnet_layer.trainable = False
    
    model = keras.Sequential([
        efficientnet_layer,
        keras.layers.Dropout(0.2),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(1000, activation='selu'),
        keras.layers.Dropout(0.2),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(1000, activation='selu'),
        keras.layers.Dropout(0.2),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=1e-2),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model
  


In [44]:
def create_finetuned_efficientnet_model(n_classes):
    efficientnet_layer = tf.keras.applications.EfficientNetB4(
            include_top= False,
            weights="imagenet")
    efficientnet_layer.trainable = False
    
    model = keras.Sequential([
        efficientnet_layer,
        keras.layers.GlobalAveragePooling2D(name = "avg_pool"),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(n_classes, activation = 'softmax')
    ])
    
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=1e-2),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [45]:
def create_unfreezed_efficientnet_model(model):
    for layer in model.layers[-12:]:
        if not isinstance(layer, keras.layers.BatchNormalization):
            layer.trainable = True
    
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [46]:
model = create_finetuned_efficientnet_model(n_classes)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 avg_pool (GlobalAveragePool  (None, 1280)             0         
 ing2D)                                                          
                                                                 
 batch_normalization_3 (Batc  (None, 1280)             5120      
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 1280)              0         
                                                                 
 dense_3 (Dense)             (None, 100)               128100    
                                                                 
Total params: 4,182,791
Trainable params: 130,660
Non-

In [47]:
callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir = 'logs'
    ), 
    tf.keras.callbacks.ModelCheckpoint(
        filepath='ckp',
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
]


history = model.fit(train_ds, epochs=10, callbacks=callbacks, validation_data=val_ds)

13/59 [=====>........................] - ETA: 46s - loss: 4.6164 - accuracy: 0.3317

In [ ]:
model = create_unfreezed_efficientnet_model(model)
history = model.fit(train_ds, epochs=10, callbacks=callbacks, validation_data=val_ds)

59/59 [==============================] - 244s 4s/step - loss: 2.3977 - accuracy: 0.5665 - val_loss: 3.4451 - val_accuracy: 0.4327


In [ ]:
model.evaluate(test_ds)

1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]

In [ ]:
model.save('empanadanet.h5')